# Connecting to camera

Never have multiple notebooks and kernels reading the video stream from a camera at the same time as they might start conflicting with each other.

[Getting Started with Videos](https://docs.opencv.org/master/dd/d43/tutorial_py_video_display.html)

In [ ]:
#
# This version requires OpenCV to be built with FFMPEG and FFMPEG plugin enabled.
# The current environment is using conda-forge OpenCV package which is not built with FFMPEG (or GStreamer) plugin enabled so 
# libopencv_videoio_ffmpeg*.so and libopencv_videoio_gstreamer*.so files are not present in ~/anaconda3/envs/python-cvcourse/lib.
# This example won't work (VideoCapture will not open).
#

import cv2

print(F'cv2 version = {cv2.__version__}')

# To find the id of the desired camera, execute
# $ v4l2-ctl --list-devices
# The output will contain the name of the camera and its matching device(s), in form /dev/videoN
# This N is the device ID
deviceId = 0

# cv2a.videoio_registry.getBackends() returns list of all available backends.
availableBackends = [cv2.videoio_registry.getBackendName(b) for b in cv2.videoio_registry.getBackends()]
print(availableBackends)

# Returns list of available backends which works via cv::VideoCapture(int index)
availableCameraBackends = [cv2.videoio_registry.getBackendName(b) for b in cv2.videoio_registry.getCameraBackends()]
print(availableBackends)

# output: ['FFMPEG', 'GSTREAMER', 'CV_IMAGES', 'CV_MJPEG']

# capture
# cap = cv2.VideoCapture(deviceId)
videoCaptureApi = cv2.CAP_ANY        # autodetect default API
#videoCaptureApi = cv2.CAP_FFMPEG
#videoCaptureApi = cv2.CAP_GSTREAMER 
#videoCaptureApi = cv2.CAP_V4L2 
#videoCaptureApi = cv2.CAP_IMAGES
#videoCaptureApi = cv2.CAP_OPENCV_MJPEG
cap = cv2.VideoCapture("/dev/video3", videoCaptureApi)

# cap is of type cv2.VideoCapture
print(f'type(cap) = {type(cap)}')

if not cap.isOpened():
    raise RuntimeError("ERROR! Unable to open camera")

try:
    # width and height of the capture is required for manipulating the image
    # # get() returns float e.g. 1080.0 and we can cast it to int via `int()`
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    heigth = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print(f'width = {width}, heigth = {heigth}')

    while True:
        # A capture is a series of images.
        # Each camera has a property called capture frame rate.
        # It denotes how frequently camera captures the frame. 
        # Its measurement unit is Frames Per Second FPS (e.g. 60 fps).
        # An image buffer is updated each time a new capture occurs.
        # A frame is a single image and we can read it from buffer. 
        # Once we get the frame, we can process it just like we processed
        # images loaded from the disk.
        #
        # tuple unpacking
        ret, frame = cap.read()

        # convert a frame into grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        cv2.imshow('frame', gray)

        # press 'q' for exit:
        #    wait 1 millisecond and check if 'q' was pressed
        #    ord() returns the integer that represents the character
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
finally:        
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
#
# This version is launching separate ffmpeg process which outputs stream into a pipe. 
# Each frame is read from a pipe in a loop and passed to OpenCV for processing and displaying.
#

import os
import tempfile
import subprocess
import cv2
import numpy as np

# FFMPEG_BIN = '/usr/bin/ffmpeg'
# To get this path execute:
#    $ which ffmpeg
FFMPEG_BIN = '/home/bojan/anaconda3/envs/python-cvcourse/bin/ffmpeg'

# to find allowed formats 
#    $ ffmpeg -f v4l2 -list_formats all -i /dev/video3
#    ...
#    [video4linux2,v4l2 @ 0x5608ac90af40] Raw: yuyv422: YUYV 4:2:2: 640x480 1280x720 960x544 800x448 640x360 424x240 352x288 320x240 800x600 176x144 160x120 1280x800
#    ...

# /home/bojan/anaconda3/envs/python-cvcourse/bin/ffmpeg -i /dev/video2 -video_size 640x480 -r 1 -pix_fmt bgr24 -vcodec rawvideo -an -sn -f v4l2
def run_ffmpeg():
    ffmpg_cmd = [
        FFMPEG_BIN,
        '-i', '/dev/video3',
        # '-framerate', '25',
        '-video_size', '640x480',
        # '-r', '1',                # framerate (fps, default is 25)
        '-pix_fmt', 'bgr24',        # opencv requires bgr24 pixel format. 'yuyv422'
        '-vcodec', 'rawvideo',
        '-an','-sn',                # disable audio processing
        # '-f', 'v4l2',
        '-f', 'image2pipe',
        '-',                        # output to go to stdout
    ]
    return subprocess.Popen(ffmpg_cmd, stdout = subprocess.PIPE, bufsize=10**8)

def run_cv_window(process):
    while True:
        # Capture frame-by-frame
        raw_image = process.stdout.read(640*480*3)
        # print(type(raw_image))
        # print(raw_image) # only for debugging but it might make browser unresponsive due to large amount of data being appended to the display in this tab
        if raw_image == b'':
            raise RuntimeError("Empty pipe")
        
        # transform the byte read into a numpy array
        frame =  np.frombuffer(raw_image, dtype='uint8')
        frame = frame.reshape((480,640,3))          # height, width, colours
        if frame is not None:
            # cv2.imshow('Video', frame) # uncomment to show the original image
            
            # example how to edit captured frame and then display it
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            cv2.imshow('frame', gray)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        process.stdout.flush()
    
    cv2.destroyAllWindows()
    process.terminate()
    print(process.poll())


def run():
    ffmpeg_process = run_ffmpeg()
    run_cv_window(ffmpeg_process)

run()

In [ ]:
#
# This version uses VLC (https://www.videolan.org/index.en-GB.html) to stream camera video
# to network via RTP protocol (https://en.wikipedia.org/wiki/Real-time_Transport_Protocol)
# and OpenCV VideoCapture to capture and modify it. This approach is good as it uses VideoCapture
# class so we can query video's properties via it. Bad thing is that image lags few seconds 
# (it's not exactly real time).
#
import cv2
import os
import subprocess
# import time

os.environ["OPENCV_FFMPEG_CAPTURE_OPTIONS"] = "rtsp_transport;udp"

# Run this in the terminal:
#    $ vlc v4l2:// :v4l2-dev=/dev/video4 :v4l2-width=640 :v4l2-height=480 --sout="#transcode{vcodec=h264,vb=800,scale=1,acodec=mp4a,ab=128,channels=2,samplerate=44100}:rtp{sdp=rtsp://:8554/live.ts}" -I dummy
# or launch vlc from python: (NOTE: running vlc v4l2 module does not work for some reason and I need to figure out why)
#
# $ which vlc
# /usr/bin/vlc
VLC_BIN = '/usr/bin/vlc'
def run_vlc():
    vlc_cmd = [
        VLC_BIN,
        # '-vvv',
        'v4l2://', 
        ':v4l2-dev=/dev/video5',
        ':v4l2-width=640', 
        ':v4l2-height=480', 
        '--sout="#transcode{vcodec=h264,vb=800,scale=1,acodec=mp4a,ab=128,channels=2,samplerate=44100}:rtp{sdp=rtsp://:8554/live.ts}"',
        #'--sout', '"#transcode{vcodec=h264,vb=800,scale=1,acodec=mp4a,ab=128,channels=2,samplerate=44100}:rtp{sdp=rtsp://:8554/live.ts}"',
        #'--sout', '#transcode{vcodec=h264,vb=800,scale=1,acodec=mp4a,ab=128,channels=2,samplerate=44100}:rtp{sdp=rtsp://:8554/live.ts}',
        #'-Idummy'
        '-I', 'dummy'
    ]
    return subprocess.Popen(vlc_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, stdin=subprocess.PIPE)
    # return subprocess.Popen(vlc_cmd, universal_newlines=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, stdin=subprocess.PIPE)
    # return subprocess.Popen(vlc_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, stdin=subprocess.PIPE, close_fds=True)

def run_vlc2():
    vlc_cmd = [
        VLC_BIN,
        'v4l2://', 
        '--v4l2-dev', '/dev/video5',
        '--v4l2-width', '640', 
        '--v4l2-height', '480', 
        '--sout="#transcode{vcodec=h264,vb=800,scale=1,acodec=mp4a,ab=128,channels=2,samplerate=44100}:rtp{sdp=rtsp://:8554/live.ts}"',
        '-I', 'dummy'
    ]
    return subprocess.Popen(vlc_cmd)

def run_vlc3():
    vlc_cmd = '/usr/bin/vlc v4l2:// :v4l2-dev=/dev/video5 :v4l2-width=640 :v4l2-height=480 --sout="#transcode{vcodec=h264,vb=800,scale=1,acodec=mp4a,ab=128,channels=2,samplerate=44100}:rtp{sdp=rtsp://:8554/live.ts}" -I dummy'
    return subprocess.Popen(vlc_cmd, shell=True)

def run_vlc4():
    vlc_cmd = [
        VLC_BIN,
        '--version',
    ]
    return subprocess.Popen(vlc_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, stdin=subprocess.PIPE)

# Uncomment this in order to try to launch vlc from python code
# vlc_proc = run_vlc4()

#while True:
#    output = vlc_proc.stdout.readline()
#    if output == '' and process.poll() is not None:
#        break
#    if output:
#        print(output.strip())
#rc = process.poll()
#print(f'rc={rc}')
    

#stdout = vlc_proc.communicate()[0]
#print(f'STDOUT:{stdout}')
#print(vlc_proc.poll())
#time.sleep(2)
#print(vlc_proc.poll())

# use ifconfig to determine the local IP address
cap = cv2.VideoCapture("rtsp://192.168.0.15:8554/live.ts")

# cap is of type cv2.VideoCapture
print(f'type(cap) = {type(cap)}')

try:
    if not cap.isOpened():
        raise RuntimeError("ERROR! Unable to open camera")
finally:
    vlc_proc.terminate()
    print(vlc_proc.poll())
        
try:
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    heigth = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print(f'width = {width}, heigth = {heigth}')

    while True:
        ret, frame = cap.read()

        # convert a frame into grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        cv2.imshow('frame', gray)

        # press 'q' for exit:
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
finally:        
    cap.release()
    cv2.destroyAllWindows()
    
vlc_proc.terminate()
print(vlc_proc.poll())

## Saving video stream to a file

[cv::VideoWriter class](https://docs.opencv.org/3.4/dd/d9e/classcv_1_1VideoWriter.html)

* filename = Name of the output video file.
* fourcc = 4-character code of codec used to compress the frames.
    * Depends on the OS. 
    * [Video Codecs by FOURCC](https://www.fourcc.org/codecs.php). 
    * [what is the codec for mp4 videos in python OpenCV - Stack Overflow](https://stackoverflow.com/questions/30103077/what-is-the-codec-for-mp4-videos-in-python-opencv)
* fps = Framerate of the created video stream (e.g. 30fps)
* frameSize = Size of the video frames `(width, height)`

In [1]:
import os
import tempfile
import subprocess
import cv2
import numpy as np

FFMPEG_BIN = '/home/bojan/anaconda3/envs/python-cvcourse/bin/ffmpeg'
width = 640
height = 480

def run_ffmpeg():
    ffmpg_cmd = [
        FFMPEG_BIN,
        '-i', '/dev/video3',
        '-video_size', f'{width}x{height}',
        '-pix_fmt', 'bgr24',        # opencv requires bgr24 pixel format. 'yuyv422'
        '-vcodec', 'rawvideo',
        '-an','-sn',                # disable audio processing
        '-f', 'image2pipe',
        '-',                        # output to go to stdout
    ]
    return subprocess.Popen(ffmpg_cmd, stdout = subprocess.PIPE, bufsize=10**8)

# cv2.VideoWriter_fourcc('m','p','4','v') can also be written as cv2.cv.CV_FOURCC(*'mp4v')
# Codec choice depends on OS:
#    Win: e.g. DIVX, ...
#    *NIX: e.g. XVID, mp4v, ...
# fps - for USB cameras choose between 20 and 30; the more frames, the larger the file
writer = cv2.VideoWriter(filename='../data/myvideo.mp4', fourcc=cv2.VideoWriter_fourcc('m','p','4','v'), fps=20, frameSize=(width, height))

def run_cv_window(process):
    while True:        
        raw_image = process.stdout.read(640*480*3)
        if raw_image == b'':
            raise RuntimeError("Empty pipe")
        frame =  np.frombuffer(raw_image, dtype='uint8')
        frame = frame.reshape((height,width,3))          # height, width, colours
        if frame is not None:
            writer.write(frame)          
            cv2.imshow('Video', frame) # we need to show display dialog so we can capture pressed keys
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        process.stdout.flush()
    
    writer.release()
    cv2.destroyAllWindows()
    process.terminate()
    print(process.poll())


def run():
    ffmpeg_process = run_ffmpeg()
    run_cv_window(ffmpeg_process)

run()

# todo: fix the bug where ffmpeg process is not terminated
# current workaround is to restart the kernel (as kernel process named 'python' is the parent process of ffmpeg).

None
